In [1]:
import pickle
import pandas as pd
import numpy as np

In [187]:
with open('../data/output/lda.pkl', 'rb') as f:
    lda_results = pickle.load(f)
f.close()    

topic_term_dists = lda_results['topic_term_dists']
doc_topic_dists = lda_results['doc_topic_dists']
doc_lengths = lda_results['doc_lengths']
vocab = lda_results['vocab']
term_frequency = lda_results['term_frequency']

In [189]:
with open('../data/output/drug_tables.pkl', 'rb') as f:
    km_results = pickle.load(f)
f.close()
km = km_results['kmeans_result']

In [190]:
km

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=30000,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [170]:
# with open('./kmeans.pkl', 'rb') as f:
#     km_results = pickle.load(f)
# f.close()    

In [110]:
with open('../data/output/dec.pkl', 'rb') as f:
    dec_results = pickle.load(f)
f.close()    

In [172]:
df = pd.read_csv('../data/output/processed.tsv', sep='\t', header=None)
df = df[1]
df.head()

0                              ['analysi', 'efficaci']
1    ['comparison', 'postop', 'ca', 'level', 'survi...
2                      ['pattern', 'diseas', 'relaps']
3    ['grade', 'advers', 'event', 'gemcitabin', 'ge...
4                    ['treatment', 'zoledron', 'acid']
Name: 1, dtype: object

In [174]:
df_ori = pd.read_csv('../data/output/origin_text.tsv', sep='\t', header=None)
df_ori = df_ori[[1]]
df_ori.head()

,1
0,Analysis of efficacy
1,Comparisons of postoperative CA19-9 levels on ...
2,Pattern of disease relapse
3,Grade 1–5 adverse events with gemcitabine alon...
4,Treatment with zoledronic acid


In [175]:
type(df_ori)

pandas.core.frame.DataFrame

In [176]:
len(corpus)

27960

In [177]:
len(df_ori)

27960

In [191]:
saved_model = {} 
# saved_model['lda_result'] = {
#         'topic_model':topic_model,
#         'corpus':corpus,
#         'dictionary':dictionary
#         }
saved_model['lda_result'] = {
        'document_topic_counts':topic_term_dists,
        'topic_word_counts':doc_topic_dists,
        'topic_counts':doc_lengths,
        'document_lengths':vocab,
        'distinct_words':term_frequency
        }

In [192]:
saved_model['kmeans_result'] = km#km_results['topic_model']

In [193]:
saved_model['dec_result'] = dec_results['label']

In [194]:
saved_model['documents'] = df_ori
saved_model['processed_docs'] = df

In [195]:
saved_model.keys()

dict_keys(['lda_result', 'kmeans_result', 'dec_result', 'documents', 'processed_docs'])

In [196]:
with open('../data/output/2_drug_tables.pkl', 'wb') as f:
    pickle.dump(saved_model, f)
f.close()

In [79]:
type(topic_model)

gensim.models.wrappers.ldamallet.LdaMallet

In [186]:
km_results['topic_model'].cluster_centers_.shape

(8, 147)

In [185]:
km_results['topic_model'].labels_

array([2, 0, 0, ..., 2, 0, 3], dtype=int32)

In [149]:
np.zeros((8, 10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [150]:
for c, n_docs in enumerate(8):
    print(c, n_docs)

TypeError: 'int' object is not iterable

In [153]:
from sklearn.feature_extraction.text import CountVectorizer

predoc = df.apply(lambda x: x[1:-1].replace("'", "").split(', '))

vect = CountVectorizer()
vect.fit([' '.join(d) for d in predoc])

tsne_data = vect.transform([' '.join(d) for d in predoc]).toarray()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [155]:
from sklearn.preprocessing import normalize

X = normalize(tsne_data, norm='l2')

In [156]:
X.shape

(27960, 9699)

In [157]:
n_docs, n_terms = X.shape

In [158]:
n_clusters = km_results['topic_model'].cluster_centers_.shape[0]

In [160]:
from collections import Counter

cluster_size = Counter(km_results['topic_model'].labels_)
cluster_size = np.asarray([cluster_size.get(c, 0) for c in range(n_clusters)])

In [201]:
with open('../data/output/drug_table.pkl', 'rb') as f:
    results = pickle.load(f)
f.close()    

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KMeans from version 0.19.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [202]:
results

{'dec_result': array([0, 4, 3, ..., 0, 2, 6]),
 'documents':           id                                           document
 0          0                               Analysis of efficacy
 1          1  Comparisons of postoperative CA19-9 levels on ...
 2          2                         Pattern of disease relapse
 3          3  Grade 1–5 adverse events with gemcitabine alon...
 4          4                     Treatment with zoledronic acid
 5          5                           Treatment with docetaxel
 6          6  Treatments ever used at relapse, at the discre...
 7          7  Worst adverse event  (grade)  reported over en...
 8          8  Chemotherapy delivery and trial drug discontin...
 9          9                                     Adverse events
 10        10  Screening effectiveness for selective and univ...
 11        11  Diagnostic effectiveness of selective versus u...
 12        12  Association between perinatal outcomes of esti...
 13        13  Age started smo

In [203]:
with open('../data/output/dec.pkl', 'rb') as f:
    dec_results = pickle.load(f)
f.close()    

In [205]:
results['dec_result'] = dec_results['label']
with open('../data/output/2_drug_table.pkl', 'wb') as f:
    pickle.dump(results, f)
f.close()